In [16]:
import os
import json
import itertools
import string
from collections import Counter
from itertools import chain

import gensim
import numpy as np
from nltk.tokenize import WordPunctTokenizer


In [17]:

# Скачиваем данные
!wget https://www.dropbox.com/s/obaitrix9jyu84r/quora.txt?dl=1 -O ./quora.txt -nc


--2024-10-30 06:53:07--  https://www.dropbox.com/s/obaitrix9jyu84r/quora.txt?dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.6.18, 2620:100:601c:18::a27d:612
Connecting to www.dropbox.com (www.dropbox.com)|162.125.6.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://www.dropbox.com/scl/fi/p0t2dw6oqs6oxpd6zz534/quora.txt?rlkey=bjupppwua4zmd4elz8octecy9&dl=1 [following]
--2024-10-30 06:53:07--  https://www.dropbox.com/scl/fi/p0t2dw6oqs6oxpd6zz534/quora.txt?rlkey=bjupppwua4zmd4elz8octecy9&dl=1
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘./quora.txt’

./quora.txt             [  <=>               ] 164.90K   353KB/s    in 0.5s    

2024-10-30 06:53:08 (353 KB/s) - ‘./quora.txt’ saved [168859]



'    });\n'

In [23]:
data = list(open("./quora.txt", encoding="utf-8"))
data[50]

"What TV shows or books help you read people's body language?\n"

In [24]:

# Токенизация текста
tokenizer = WordPunctTokenizer()
data_tok = [
    tokenizer.tokenize(
        line.translate(str.maketrans("", "", string.punctuation)).lower()
    )
    for line in data
]
data_tok = [x for x in data_tok if len(x) >= 3]


In [25]:

# Убираем редкие слова
min_count = 5
vocabulary_with_counter = Counter(chain.from_iterable(data_tok))
word_count_dict = {word: count for word, count in vocabulary_with_counter.items() if count >= min_count}
vocabulary = set(word_count_dict.keys())
data_tok = [[word for word in text if word in vocabulary] for text in data_tok]


In [26]:

# Обучение модели Word2Vec
model = gensim.models.Word2Vec(
    sentences=data_tok,
    vector_size=32,
    window=5,
    min_count=5,
    sg=1,
    negative=15
)

In [27]:
# Пример с другими словами
similarity_computer_laptop = model.wv.similarity("computer", "laptop")
similarity_computer_desk = model.wv.similarity("computer", "desk")
print(f"Similarity between 'computer' and 'laptop': {similarity_computer_laptop}")
print(f"Similarity between 'computer' and 'desk': {similarity_computer_desk}")


Similarity between 'computer' and 'laptop': 0.6427711844444275
Similarity between 'computer' and 'desk': 0.5514743328094482


In [28]:
# Чтение слов из words_subset.txt
assert os.path.exists("words_subset.txt"), "Please, download `words_subset.txt` and place it in the working directory"
with open("words_subset.txt") as iofile:
    selected_words = iofile.read().split("\n")

# Генерация эмбеддингов для `submission_dict.json`
def get_matrix_for_selected_words(selected_words, model):
    word_vectors = []
    for word in selected_words:
        if word in model.wv:
            vector = model.wv[word].tolist()
        else:
            vector = [0.0] * model.vector_size
        word_vectors.append(vector)
    return word_vectors

word_vectors = get_matrix_for_selected_words(selected_words, model)

# Сохранение вектора слов в файл JSON
with open("submission_dict.json", "w") as iofile:
    json.dump(word_vectors, iofile)
print("File saved to `submission_dict.json`")

File saved to `submission_dict.json`
